## TE Connect CNN-LSTM Attempt

Author: Alexiy Buynitsky

In [18]:
import pandas as pd
from matplotlib import pyplot as plt
from pandas import DataFrame

df = pd.read_csv("te_ai_cup_sales_forecasting_data.csv")
print("num rows:", len(df))
print("num cols:", df.shape[1])
df["year_week_ordered"] = df['fiscal_year_historical'] + df['fiscal_week_historical']
df.sort_values(by=['fiscal_year_historical'] + ['fiscal_week_historical'], inplace=True)
df.reset_index(drop=True, inplace=True)
u_prod_code = df["product_line_code"].unique()
print(u_prod_code)
display(df.head())

num rows: 522698
num cols: 10
['E77' '053' '202' 'ABP' 'AC5' '607' 'AC6' '088' 'H98' 'DK4' 'N65' '372'
 'FG2' '251' '208' 'F49' 'H90' 'J67' 'J63' '073' '351' '050' 'K70' '301'
 'J37' 'A13' 'AQ9' '038' '025' '440' '481' '145' '230' 'K85' 'JB3' '547'
 '239' '912' '281' '231' 'LT0' '214' 'JF1' 'A28' '567' 'B91' '438' '430'
 'K00' 'J89' 'E31' 'Z27' '291' 'Z83' 'G60' '518' '472' 'F52' 'G59' '042'
 'N09' '323' '715' 'A05' '072' '397' '957' 'J25' '217' '014' '224' 'G49'
 '344' '207' 'AP2' '002' '529' '077' 'E82' '835' 'LV4' '728' '890' '032'
 '757' 'E79' '097' 'Z34' '057' 'F38' 'Z43' 'F83' 'F82' 'G74' 'AG7' '685'
 '977' '009' 'A92' '712' 'F50' '001' 'JH4' '402' '124' '023' '961' '313'
 'G61' 'F48' '056' 'G41' '232' '766' '519' 'J12' '243' 'LV1' '204' 'F41'
 '450' '516' 'W02' 'Z40' '080' '289' '036' '394' '150' 'JC1' 'J46' 'AH8'
 '046' 'F40' '716' 'G30' 'B77' 'JJ4' 'C19' 'DA8' 'K80' '327' 'E83' 'F46'
 '350' 'J45' '748' 'Z77' 'N73' '914' 'JE2' '085' 'JC3' '312' '127' 'J96'
 '760' '076' 'K18' 'N

,fiscal_year_historical,fiscal_quarter_historical,fiscal_month_historical,fiscal_week_historical,business_unit_group_name,company_region_name_level_1,product_line_code,product_line_name,sales_quantity,sales_amount,year_week_ordered
0,2015,1,1,1,Energy,EMEA,E77,LV/MV Surge Arresters,1641.00,126789.47,2016
1,2015,1,1,1,Channel - Data and Devices,Asia Pacific & ANZ,053,Memory Sockets,14417.92,17379.52,2016
2,2015,1,1,1,Channel - Industrial,EMEA,202,PLASTI-GRIP Terminals,142820.40,19666.38,2016
3,2015,1,1,1,Industrial,Americas,ABP,Specific terminal blocks,14528.00,-12089.77,2016
4,2015,1,1,1,Data and Devices,EMEA,AC5,"Antennas, Custom Others",34400.00,30357.47,2016


## Splitting the Data based on Specific Paramters

In [29]:
def get_raw_data(filtered_df,business_unit_group_name=None,company_region_name_level1=None,product_line_code=None):

    if business_unit_group_name is not None:
        filtered_df = filtered_df[filtered_df['business_unit_group_name'] == business_unit_group_name]
    if company_region_name_level1 is not None:
        filtered_df = filtered_df[filtered_df['company_region_name_level_1'] == company_region_name_level1]
    if product_line_code is not None:
        filtered_df = filtered_df[filtered_df['product_line_code'] == product_line_code]

    if len(filtered_df)==0:
        raise Exception("ERROR: empty dataset")
    return filtered_df

#raw_df = get_raw_data(df,company_region_name_level1= "EMEA")
raw_df = get_raw_data(df)
display(raw_df.head())

,fiscal_year_historical,fiscal_quarter_historical,fiscal_month_historical,fiscal_week_historical,business_unit_group_name,company_region_name_level_1,product_line_code,product_line_name,sales_quantity,sales_amount,year_week_ordered
0,2015,1,1,1,Energy,EMEA,E77,LV/MV Surge Arresters,1641.00,126789.47,2016
1,2015,1,1,1,Channel - Data and Devices,Asia Pacific & ANZ,053,Memory Sockets,14417.92,17379.52,2016
2,2015,1,1,1,Channel - Industrial,EMEA,202,PLASTI-GRIP Terminals,142820.40,19666.38,2016
3,2015,1,1,1,Industrial,Americas,ABP,Specific terminal blocks,14528.00,-12089.77,2016
4,2015,1,1,1,Data and Devices,EMEA,AC5,"Antennas, Custom Others",34400.00,30357.47,2016


In [30]:
grouped_df = df.groupby("product_line_code")

In [31]:
for name, group in grouped_df:
    print(name)
    print(len(group))
    display(group.head())
    break

001
833


,fiscal_year_historical,fiscal_quarter_historical,fiscal_month_historical,fiscal_week_historical,business_unit_group_name,company_region_name_level_1,product_line_code,product_line_name,sales_quantity,sales_amount,year_week_ordered
108,2015,1,1,1,Channel - Industrial,Asia Pacific & ANZ,001,Miscellaneous AMPMODU,305278.80,33869.70,2016
1043,2015,1,1,1,Industrial,Asia Pacific & ANZ,001,Miscellaneous AMPMODU,513152.00,76599.06,2016
1299,2015,1,1,2,Channel - Industrial,Asia Pacific & ANZ,001,Miscellaneous AMPMODU,396862.44,44030.61,2017
2258,2015,1,1,2,Industrial,Asia Pacific & ANZ,001,Miscellaneous AMPMODU,179474.00,30120.36,2017
2484,2015,1,1,3,Channel - Industrial,Asia Pacific & ANZ,001,Miscellaneous AMPMODU,396862.44,44030.61,2018


## Preparing the Data for Training